# Create Layer for Lambda

## Version 1 

see <https://nordcloud.com/lambda-layers-for-python-runtime/>

In [ ]:
!rm -rf build
PY_DIR='build/python/lib/python3.7/site-packages'
!mkdir -p {PY_DIR}
!pip install -r requirements.txt --no-deps -t {PY_DIR}

In [ ]:
!chmod -R go+r build/

In [ ]:
!find build/ -type d -exec chmod 755 {} \;

In [ ]:
!rm sklearn.zip
!cd {PY_DIR} && zip -r9 ../../../../../sklearn.zip .   

## Version 2 

<https://medium.com/@qtangs/creating-new-aws-lambda-layer-for-python-pandas-library-348b126e9f3e>

In [ ]:
import os
PKG_DIR="python"
CWD = os.getcwd()


!rm -rf {PKG_DIR} && mkdir -p {PKG_DIR}

!docker run --rm -v {CWD}:/foo -w /foo lambci/lambda:build-python3.7 \
    pip install -r requirements.txt --no-deps -t {PKG_DIR}

# Upload to AWS

In [ ]:
!aws s3 mb s3://lambda-scikit-bucket --region us-east-1 --profile dev

In [ ]:
!aws s3 cp ./sklearn.zip s3://lambda-scikit-bucket/ --profile dev

In [40]:
!aws s3 ls s3://lambda-scikit-bucket/ --profile dev

2018-12-23 05:35:03   53817870 sklearn.zip


In [ ]:
!aws lambda publish-layer-version \
                        --region us-east-1 --layer-name sklearn \
                        --compatible-runtimes python3.7\
                        --content S3Bucket=lambda-scikit-bucket,S3Key=sklearn.zip --profile dev

In [43]:
!aws s3 cp s3://lambda-scikit-bucket/sklearn.zip . --profile dev

download: s3://lambda-scikit-bucket/sklearn.zip to ./sklearn.zip   
